# Toronto Data Segmentation and Clustering
By Jonathan Dietrich

## Part 1: Get the data and prepare it

In [1]:
import numpy as np
import pandas as pd

Put the toronto postal code table from the Wikipedia page into a Pandas Dataframe and remove the postal codes with no assigned burrow. 

In [2]:
table_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_postal = pd.read_html(table_url, header=0)[0]
df_postal = df_postal[df_postal.Borough != 'Not assigned']
df_postal.reset_index(inplace=True)
del df_postal['index']
assert df_postal['Postal Code'].is_unique
df_postal.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df_postal.shape

(103, 3)

## Part 2: Get the coordinates for the postal codes

In [4]:
# ! pip install geocoder
import geocoder

Get csv file with coordinates (geocoder API was not working).

In [5]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.set_index('Postal Code', inplace=True)
df_coords.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Put coordinates in df_postal by merging the dataframes.

In [6]:
df = df_postal.join(df_coords, on='Postal Code', how='left')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3: Explore and Cluster Neighborhoods

In [7]:
import json # library to handle JSON files

#! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#! pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Only use boroughs that contain the word Toronto.

In [8]:
toronto_df = df[df['Borough'].str.contains("Toronto")]
toronto_df.reset_index(inplace=True)
del toronto_df['index']
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Count boroughs and neighborhoods.

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


Show neighborhoods on map.

In [10]:
# get mean latitude and longitude
latitude_mean = toronto_df.Latitude.mean()
longitude_mean = toronto_df.Longitude.mean()

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude_mean, longitude_mean], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare credentials and version in separate config file that is not in the git repository so my credentials are not published.

In [11]:
from foursquare_config import *

### 3a Explore first neighborhood

Let's explore the first postal code in our dataframe.

Get the first row.

In [12]:
toronto_df.loc[0, :]

Postal Code                            M5A
Borough                   Downtown Toronto
Neighbourhood    Regent Park, Harbourfront
Latitude                           43.6543
Longitude                         -79.3606
Name: 0, dtype: object

Get the top 100 venues that are in the neighborhood Regend Park using a foursquare get request.

In [13]:
# type your answer here
search_query = toronto_df.loc[0, 'Neighbourhood'].split(', ')[0]
neighborhood_latitude = toronto_df.loc[0, 'Latitude']
neighborhood_longitude = toronto_df.loc[0, 'Longitude']
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)

Send the get request and examine the results.

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4126283649c121d43e8f97'},
 'response': {'venues': [{'id': '4eef45335c5c794ad4305a1d',
    'name': 'Regent Park School of Music',
    'location': {'address': '534 Queen Street East',
     'lat': 43.65691161146316,
     'lng': -79.3571683258484,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65691161146316,
       'lng': -79.3571683258484}],
     'distance': 406,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['534 Queen Street East', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e5931735',
      'name': 'Music Venue',
      'pluralName': 'Music Venues',
      'shortName': 'Music Venue',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1598105428',
    'hasPerk': False},
   {'id': '56cde159498e8978c386d953',
    'name': 'Regent P

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Check how the venues data is structured

In [16]:
results['response']['venues']

[{'id': '4eef45335c5c794ad4305a1d',
  'name': 'Regent Park School of Music',
  'location': {'address': '534 Queen Street East',
   'lat': 43.65691161146316,
   'lng': -79.3571683258484,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65691161146316,
     'lng': -79.3571683258484}],
   'distance': 406,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['534 Queen Street East', 'Toronto ON', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1e5931735',
    'name': 'Music Venue',
    'pluralName': 'Music Venues',
    'shortName': 'Music Venue',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1598105428',
  'hasPerk': False},
 {'id': '56cde159498e8978c386d953',
  'name': 'Regent Park Employment Services',
  'location': {'lat': 43.65785,
   'lng': -79.36189,
   'labeledLatLngs': [{'label': 'display', 'lat': 43.65785

Clean json structure into pandas dataframe.

In [17]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Regent Park School of Music,Music Venue,43.656912,-79.357168
1,Regent Park Employment Services,Government Building,43.657850,-79.361890
2,Regent Park / Duke of York Junior Public School,Middle School,43.657764,-79.363933
3,Regent Park Focus,Community Center,43.658712,-79.364197
4,Regent Park South Rink,Skating Rink,43.659131,-79.364923


How many venues were returned by foursquare?

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


### 3b Explore all neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a dataframe called toronto_venues.

In [20]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Remove venues with the venue category neighborhood since I'm interested in venues and not in neighborhoods. This also avoids problems with duplicate column names later.

In [21]:
display(toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'])
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']
toronto_venues.reset_index(inplace=True, drop=True)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
267,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
414,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
520,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
958,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Check the size of the resulting dataframe.

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1631, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood.

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


### 3c Analyze each neighborhood

Do one-hot encoding of all venues into venue categories.

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot = pd.concat([toronto_venues['Neighborhood'], toronto_onehot], axis=1)
print(toronto_onehot.shape)
toronto_onehot.head()

(1631, 235)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.041667
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.015873,0.0,0.015873


Check new size.

In [27]:
toronto_grouped.shape

(39, 235)

Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3         Cheese Shop  0.04
4      Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3          Bakery  0.08
4     Yoga Studio  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1  Gym / Fitness Center  0.06
2               Brewery  0.06
3            Skate Park  0.06
4            Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3              Airport  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                ve

Let's put that into a pandas dataframe

Function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Café,Beer Bar,Cocktail Bar,Italian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boutique,Sculpture Garden,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Salad Place,Ramen Restaurant,Portuguese Restaurant


### 3d Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([9, 9, 1, 4, 9, 1, 9, 9, 1, 7])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
toronto_merged = toronto_df.copy()
toronto_merged.rename(columns={'Neighbourhood' : 'Neighborhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9,Coffee Shop,Yoga Studio,Fried Chicken Joint,Restaurant,Portuguese Restaurant,Persian Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,9,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,9,Coffee Shop,Café,Cocktail Bar,Restaurant,American Restaurant,Italian Restaurant,Diner,Lingerie Store,Department Store,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Pub,Health Food Store,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude_mean, longitude_mean], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters that have more than one neighborhood in them.

In [35]:
cluster_counts = toronto_merged['Cluster Labels'].value_counts()
for cn in [c for c in range(kclusters) if cluster_counts[c]>1]:
    print('Cluster ' + str(cn))
    display(toronto_merged.loc[toronto_merged['Cluster Labels'] == cn, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]])

Cluster 1


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Grocery Store,Café,Park,Diner,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Coffee Shop
9,West Toronto,1,Pharmacy,Bakery,Gym,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant,Pizza Place,Brewery
12,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Pub,Indian Restaurant,Spa,Caribbean Restaurant
15,East Toronto,1,Park,Sushi Restaurant,Board Shop,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pizza Place,Pet Store
22,West Toronto,1,Café,Mexican Restaurant,Park,Bakery,Thai Restaurant,Fast Food Restaurant,Discount Store,Bar,Flea Market,Fried Chicken Joint
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Park,Donut Shop,Liquor Store,Burger Joint,Metro Station,Indian Restaurant,Flower Shop
26,Central Toronto,1,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Toy / Game Store,Greek Restaurant
28,West Toronto,1,Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Sushi Restaurant,Yoga Studio,Gourmet Shop,Restaurant,Latin American Restaurant
31,Central Toronto,1,Coffee Shop,Pub,Sushi Restaurant,Restaurant,Bank,Supermarket,Sports Bar,Fried Chicken Joint,Pizza Place,Bagel Shop
35,Downtown Toronto,1,Pizza Place,Coffee Shop,Café,Restaurant,Pub,Bakery,Park,Italian Restaurant,Market,Farmers Market


Cluster 9


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,9,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,9,Coffee Shop,Yoga Studio,Fried Chicken Joint,Restaurant,Portuguese Restaurant,Persian Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant
2,Downtown Toronto,9,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Hotel
3,Downtown Toronto,9,Coffee Shop,Café,Cocktail Bar,Restaurant,American Restaurant,Italian Restaurant,Diner,Lingerie Store,Department Store,Beer Bar
5,Downtown Toronto,9,Coffee Shop,Farmers Market,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Café,Beer Bar,Cocktail Bar,Italian Restaurant
6,Downtown Toronto,9,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Salad Place,Ramen Restaurant,Portuguese Restaurant
8,Downtown Toronto,9,Coffee Shop,Café,Gym,Hotel,Clothing Store,Restaurant,Bar,Thai Restaurant,Steakhouse,Pizza Place
10,Downtown Toronto,9,Coffee Shop,Aquarium,Restaurant,Café,Hotel,Scenic Lookout,Brewery,Italian Restaurant,Fried Chicken Joint,Sporting Goods Shop
11,West Toronto,9,Bar,Coffee Shop,Asian Restaurant,Restaurant,Café,Men's Store,Vietnamese Restaurant,Ice Cream Shop,French Restaurant,Gift Shop
13,Downtown Toronto,9,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Salad Place,Concert Hall
